# Georeferenced maps in the State Library of Victoria's collections

This dashboard provides information on digitised maps from the [State Library of Victoria's collection](https://www.slv.vic.gov.au/search-discover/explore-collections-format/maps) that have been georeferenced using [AllMaps](https://allmaps.org).

In [18]:
import pandas as pd
import altair as alt
import ipywidgets as widgets
from ipyleaflet import Map, Polygon
from ast import literal_eval
from urllib.parse import quote_plus
import arrow
from IPython.display import Markdown

In [29]:
df = pd.read_csv("georeferenced_maps.csv", parse_dates=["modified", "created"])
df["geometry"] = df["geometry"].apply(literal_eval)

now = arrow.utcnow().to('Australia/Melbourne').format("h.mA on D MMMM YYYY")

total_maps = df.shape[0]
total_manifests = df["ie_id"].nunique()

widgets.HTML(f"<p style='border: 1px solid; padding: 20px; font-size: 1.5rem;' >As of {now}, <b>{total_maps} maps</b> have been georeferenced from {total_manifests} collection items.</p>")

HTML(value="<p style='border: 1px solid; padding: 20px; font-size: 1.5rem;' >As of 1.4PM on 29 October 2025, <…

In [30]:
Markdown("## Locations of georeferenced maps")

## Locations of georeferenced maps

Click on a highlighted area for more information.

In [32]:
default_layout = widgets.Layout(width="100%", height="600px")
m = Map(
    center=(-37.815, 144.965),
    zoom=7,
    scroll_wheel_zoom=True,
    layout=default_layout,
)

for geomap in df.sort_values("area", ascending=False).itertuples():
    coords = [(c[1], c[0]) for c in geomap.geometry["coordinates"][0]]
    polygon = Polygon(
        locations=coords,
        color="blue",
        fill_color="blue",
        weight=1
    )
    html = ""
    html += f"<p style='line-height: 1.2;'><b>{geomap.manifest_title}</b></p><p style='line-height: 1.2;'>{geomap.map_title}</p>"
    html += f"<img src='{geomap.image_id}/full/!200,200/0/default.jpg'>"
    html += f"<p style='line-height: 1.2;'><a href='https://viewer.allmaps.org/?url={geomap.allmaps_map_id}'>View in the AllMaps viewer</a><br>"
    html += f"<a href='https://viewer.slv.vic.gov.au/?entity={geomap.ie_id}&file={geomap.fl_id}&mode=browse'>View in the SLV viewer</a></p>"
    polygon.popup = widgets.HTML(html)
    m.add(polygon)
m

Map(center=[-37.815, 144.965], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

In [33]:
Markdown("## Latest georeferenced maps")

## Latest georeferenced maps

Here are the 20 maps most recently created or modified. You can also [explore the full list using Datasette](https://glam-workbench.net/datasette-lite/?csv=https%3A%2F%2Fgithub.com%2Fwragge%2Fslv-allmaps%2Fblob%2Fmain%2Fgeoreferenced_maps.csv&install=datasette-homepage-table&fts=manifest_title%2Cmap_title).

In [34]:
df["allmaps_link"] = df["allmaps_map_id"].apply(lambda x: f"<a href='https://viewer.allmaps.org/?url={x}'>View in the AllMaps viewer</a>")
df["slv_link"] = df.apply(lambda x: f"<a href='https://viewer.slv.vic.gov.au/?entity={x["ie_id"]}&file={x["fl_id"]}&mode=browse'>View in the SLV viewer</a>", axis=1)
df["thumbnail"] = df["image_id"].apply(lambda x: f"<img src='{x}/full/!100,100/0/default.jpg'>")

In [35]:
df[["thumbnail", "manifest_title", "map_title", "modified", "allmaps_link", "slv_link"]][:20].style.set_properties(**{'text-align': 'left'}).set_table_styles([dict(selector='th', props=[('text-align', 'left')])]).hide()

thumbnail,manifest_title,map_title,modified,allmaps_link,slv_link
,Melbourne and its suburbs,Map 1,2025-10-29 01:23:51.859000+00:00,View in the AllMaps viewer,View in the SLV viewer
,"H.G. de Gruchy & Co.'s parish map Colonial Map Depôt, 21 Elizabeth St., Melbourne.",Map 1,2025-10-27 23:35:36.492000+00:00,View in the AllMaps viewer,View in the SLV viewer
,1912 Moultons directory of streets for Melbourne and suburbs Moulton's directory of streets for Melbourne and suburbs.,Map Caulfield,2025-10-26 00:18:55.260000+00:00,View in the AllMaps viewer,View in the SLV viewer
,1912 Moultons directory of streets for Melbourne and suburbs Moulton's directory of streets for Melbourne and suburbs.,Map Carlton and Parkville,2025-10-11 05:47:44.079000+00:00,View in the AllMaps viewer,View in the SLV viewer
,1912 Moultons directory of streets for Melbourne and suburbs Moulton's directory of streets for Melbourne and suburbs.,Map Camberwell and Canterbury,2025-10-11 05:38:33.722000+00:00,View in the AllMaps viewer,View in the SLV viewer
,1912 Moultons directory of streets for Melbourne and suburbs Moulton's directory of streets for Melbourne and suburbs.,Map City of Brunswick,2025-10-11 05:19:05.314000+00:00,View in the AllMaps viewer,View in the SLV viewer
,1912 Moultons directory of streets for Melbourne and suburbs Moulton's directory of streets for Melbourne and suburbs.,Map Brighton,2025-10-11 05:11:54.608000+00:00,View in the AllMaps viewer,View in the SLV viewer
,1912 Moultons directory of streets for Melbourne and suburbs Moulton's directory of streets for Melbourne and suburbs.,Map North Fitzroy,2025-10-11 05:04:28.772000+00:00,View in the AllMaps viewer,View in the SLV viewer
,1912 Moultons directory of streets for Melbourne and suburbs Moulton's directory of streets for Melbourne and suburbs.,Map Box Hill,2025-10-11 04:28:52.281000+00:00,View in the AllMaps viewer,View in the SLV viewer
,City of South Melbourne and section of Port Melbourne detail-fire-survey,Map 2,2025-10-08 12:11:16.504000+00:00,View in the AllMaps viewer,View in the SLV viewer


In [25]:
Markdown("## Georeferencing progress")

## Georeferencing progress

In [26]:
alt.Chart(df).mark_bar().encode(
    x=alt.X("yearmonthdate(created):T").axis(format='%e %b %y').title(None),
    y=alt.Y("count():Q").title("number of maps")
).properties(width="container", title="maps georeferenced per day", padding=40)

alt.Chart(...)

In [27]:
alt.Chart(df).transform_timeunit(
    date='yearmonthdate(created)'
).transform_window(
    frame=[None, 0],
    cumulative_count="count()",
    sort=[{"field": "date"}],
).mark_area().encode(
    x=alt.X("date:T").axis(format='%e %b %y').title(None),
    y=alt.Y("cumulative_count:Q").title("number of maps").stack(None)
).properties(width="container", title="total maps georeferenced over time", padding=40)

alt.Chart(...)